In [ ]:
%pip install tensorflow

In [ ]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow.keras import applications, Input, Model, layers, optimizers, losses
from tensorflow.keras.utils import image_dataset_from_directory

# Global Configuration

- `DATASET_DIR`: The directory where the dataset is stored. Each subdirectory in this folder represents a separate label.
- `NUM_LABELS`: Automatically computed from the number of subdirectories in `DATASET_DIR`. This makes the notebook flexible and automatically adjusts if you add or remove label folders.
- `BATCH_SIZE` and `IMG_SIZE`: Define the batch size and target input image size (224×224) used during training.
- Training configuration parameters such as `INITIAL_EPOCHS`, `FINE_TUNE_EPOCHS`, and `FINE_TUNE_AT` control the number of training epochs and the point at which fine-tuning begins.
- Learning rate parameters (`BASE_LEARNING_RATE`, `FINE_TUNE_LEARNING_RATE`) are defined to configure the optimizer.

Feel free to change these variables to match your dataset, desired model complexity, or hardware constraints.



In [ ]:
# Path to dataset (each subdirectory represents a label)
DATASET_DIR = "dataset/"

# Automatically fetch the number of labels based on subdirectories.
# Only count directories (ignore files).
NUM_LABELS = len([d for d in sorted(os.listdir(DATASET_DIR)) if os.path.isdir(os.path.join(DATASET_DIR, d))])
print(f"Detected {NUM_LABELS} label(s) in {DATASET_DIR}")

In [ ]:
# Global configuration variables
BATCH_SIZE = 50                # Batch size for training/validation
IMG_SIZE = (224, 224)          # Input image dimensions

# Training epochs
INITIAL_EPOCHS = 10
FINE_TUNE_EPOCHS = 5
TOTAL_EPOCHS = INITIAL_EPOCHS + FINE_TUNE_EPOCHS

# Fine-tuning configuration:
FINE_TUNE_AT = 120

# Learning rates
BASE_LEARNING_RATE = 0.001
FINE_TUNE_LEARNING_RATE = 0.1 * BASE_LEARNING_RATE

# Model Building and Preprocessing Functions

These cells contain two functions:

1. **`get_model(num_labels)`**  
   - Builds a custom classifier using MobileNetV2 as the base model (pre-trained on ImageNet, with its top layers removed).
   - Adds a global average pooling layer, dropout, and a Dense layer with softmax activation for classification.
   - The number of output nodes (classes) is set dynamically based on `num_labels`, which is determined by your dataset structure.
   
2. **`preprocess(image, label)`**  
   - Applies the standard preprocessing for MobileNetV2, converting input images from [0,255] to [-1,1].
   
These functions are the foundation for model creation. You can modify or extend them if you need a different architecture or preprocessing pipeline.


In [ ]:
def preprocess(image, label):
    # Use MobileNetV2 preprocessing (converts from [0,255] to [-1,1])
    image = applications.mobilenet_v2.preprocess_input(image)
    return image, label

In [ ]:
def get_model(num_labels=NUM_LABELS):
    # Load the MobileNetV2 base model without the top layers.
    base_model = applications.MobileNetV2(
        alpha=0.35,
        input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    inputs = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_labels, activation='softmax')(x)
    model = Model(inputs, outputs)

    model.summary()
    return model

In [ ]:
# Create training and validation datasets from the directory.
train_dataset = image_dataset_from_directory(
    DATASET_DIR,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    validation_split=0.2,
    subset='training',
    seed=42,
).map(preprocess)

validation_dataset = image_dataset_from_directory(
    DATASET_DIR,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    validation_split=0.2,
    subset='validation',
    seed=42,
).map(preprocess)

# Model Training and Fine-Tuning

Here onwards we perform both initial training and fine-tuning of the model.

1. **Initial Training:**  
   - The model is built and compiled with an Adam optimizer and Sparse Categorical Crossentropy loss.
   - The model is trained for a set number of epochs (`INITIAL_EPOCHS`) using the prepared training and validation datasets.

2. **Fine-Tuning:**  
   - The base model is unfrozen partially (layers after index `FINE_TUNE_AT` are set to trainable) for fine-tuning.
   - The learning rate is reduced during fine-tuning (`FINE_TUNE_LEARNING_RATE`).
   - The model is recompiled and fine-tuned for additional epochs (`FINE_TUNE_EPOCHS`).

Finally, the model is evaluated on the validation dataset and saved in both Keras and SavedModel formats. Adjust these parameters as needed for your dataset and target performance.


In [ ]:
# Build and compile the initial model.
model = get_model(num_labels=NUM_LABELS)
model.compile(optimizer=optimizers.Adam(learning_rate=BASE_LEARNING_RATE),
              loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

print("Training the classifier...")
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=INITIAL_EPOCHS)

In [ ]:
# Fine-tuning: unfreeze the base model layers starting from FINE_TUNE_AT.
base_model = model.layers[1]
base_model.trainable = True
print("Number of layers in the base model:", len(base_model.layers))

for layer in base_model.layers[:FINE_TUNE_AT]:
    layer.trainable = False

model.compile(optimizer=optimizers.Adam(learning_rate=FINE_TUNE_LEARNING_RATE),
              loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

print(f"Fine-tuning the base model (layer {FINE_TUNE_AT} onwards)...")
history_fine = model.fit(
    train_dataset,
    epochs=TOTAL_EPOCHS,
    initial_epoch=history.epoch[-1],
    validation_data=validation_dataset,
)

In [ ]:
loss, accuracy = model.evaluate(validation_dataset)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Save the final model.
model.save('mobilenet_custom_model.keras')
model.export('mobilenet_custom_model_saved_model')

# Representative Dataset Generator for Quantization

This cell defines `representative_data_gen()`, which is crucial for INT8 quantization of your model.

- It automatically detects all label subdirectories within your dataset (so it works for any number of labels).
- For each image, it:
  - Loads and resizes the image to `IMG_SIZE`.
  - Converts the image to a numpy array and normalizes it to the range [-1,1] (as required by MobileNetV2).
  - Generates several variants of each image (using brightness and contrast adjustments) to better calibrate the quantization process.
  
The representative dataset is used by the TFLite converter to calibrate the quantization parameters, ensuring that the quantized model performs well on real-world data.


In [ ]:
def representative_data_gen():
    from tensorflow.keras.preprocessing.image import load_img, img_to_array
    # List all subdirectories in the dataset directory.
    label_dirs = [d for d in sorted(os.listdir(DATASET_DIR)) if os.path.isdir(os.path.join(DATASET_DIR, d))]

    # Optionally, limit to NUM_LABELS if defined (otherwise, use all subdirectories)
    if NUM_LABELS and len(label_dirs) > NUM_LABELS:
        label_dirs = label_dirs[:NUM_LABELS]

    # Iterate over each label directory.
    for label in label_dirs:
        label_path = os.path.join(DATASET_DIR, label)
        # List all image files in this directory.
        files = sorted(os.listdir(label_path))
        for f in files:
            img_path = os.path.join(label_path, f)
            img = load_img(img_path, target_size=IMG_SIZE)
            img_array = img_to_array(img).astype(np.float32)
            # Normalize to [-1,1]
            img_normalized = img_array / 127.5 - 1.0

            variants = [img_normalized]

            # Create brightness variants.
            for brightness in [0.8, 1.2]:
                variant = tf.image.adjust_brightness(img_array, brightness - 1)
                variant = variant / 127.5 - 1.0
                variants.append(variant.numpy())

            # Create contrast variants.
            for contrast in [0.8, 1.2]:
                variant = tf.image.adjust_contrast(img_array, contrast)
                variant = variant / 127.5 - 1.0
                variants.append(variant.numpy())

            # Yield each variant as a batch.
            for variant in variants:
                yield [np.expand_dims(variant, axis=0).astype(np.float32)]


# Model Quantization to TFLite INT8

In this final cell, we convert the trained model (SavedModel format) into a fully quantized TFLite model using INT8 precision.

Key steps include:
- **Optimizations:** Enabling default optimizations and setting the converter to use INT8 operations exclusively.
- **Representative Dataset:** The previously defined `representative_data_gen()` is provided to calibrate the quantization parameters.
- **Experimental Settings:**  
  - `_experimental_new_quantizer` is set to True.
  - `_experimental_disable_per_channel` is set to False to allow per-channel quantization.
- **Input/Output Types:** The converter is set to use `tf.int8` for both inference input and output.

After conversion, the quantized model is written to disk as `human_detect_experiment.tflite`. This file is ready for deployment on an embedded device running TFLite Micro.

Feel free to adjust these parameters if you have specific quantization requirements.


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("mobilenet_custom_model_saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.target_spec.supported_types = [tf.int8]
converter._experimental_new_quantizer = True
converter._experimental_disable_per_channel = False
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

print("Converting model to TFLite...")
tflite_model = converter.convert()

with open("mobilenet_custom_model.tflite", "wb") as f:
    f.write(tflite_model)
print("Model exported to mobilenet_custom_model.tflite")